In [ ]:
from pylab import *

ion()

import os
import time

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output

# import cv2
# from PyQt5 import QtCore, QtGui, QtWidgets
# from PyQt5.QtCore import *
# from PyQt5.QtGui import *

%matplotlib inline

In [ ]:
mm_config = os.path.join(
    os.environ["NOTEBOOK_DIR"], "../../microscope/config/DE_config.cfg"
)
logfile = os.path.join(os.environ["NOTEBOOK_DIR"], "micromanager.log")
micromanager_dir = os.environ["MICROMANAGER_DIR"]

In [ ]:
os.chdir(micromanager_dir)

In [ ]:
import MMCorePy

In [ ]:
mmc = MMCorePy.CMMCore()
mmc.loadSystemConfiguration(mm_config)
mmc.setPrimaryLogFile(logfile)

mmc.getLoadedDevices()

### Live View

In [ ]:
def liveview(camera, img_size=(12, 12), low=None, high=None):  # W,interval=0.5):
    mmc.setCameraDevice(camera)
    while True:
        try:
            while mmc.deviceBusy(camera):
                time.sleep(0.005)
            mmc.snapImage()
            im1 = mmc.getImage()
            clear_output(wait=True)
            plt.figure(figsize=img_size)
            if low == None or high == None:
                plt.imshow(im1, interpolation="None", cmap="gray")
            else:
                plt.imshow(im1, interpolation="None", cmap="gray", vmin=low, vmax=high)
            plt.show()
        except KeyboardInterrupt:
            break
    while mmc.deviceBusy(camera):
        time.sleep(0.01)


#         slp = interval - (time.time() - ti)
#         time.sleep(slp)

In [ ]:
def scale_img(img, scale):
    width = int(img.shape[1] * scale)
    height = int(img.shape[0] * scale)
    dim = (width, height)
    return cv2.resize(img, dim, interpolation=cv2.INTER_AREA)


def disp_info(img):
    clear_output(wait=True)
    print("mean image intensity: " + str(int(np.mean(img))))
    print("XY position: " + str(mmc.getXYPosition()))
    print("Z position: " + str(mmc.getPosition("ZDrive")))


# def widget_test():
#     app = QtWidgets.QApplication.instance()
#     if app is None:
#         app = QtWidgets.QApplication(sys.argv)
#     else:
#         print('QApplication instance already exists: %s' % str(app))

#     window = QtWidgets.QWidget()
#     window.setGeometry(100, 100, 500, 300)
#     window.setWindowTitle("Image Viewer")

#     label = QtWidgets.QLabel()
#     label.setAlignment(Qt.AlignCenter)

#     def update_label():
#         label.setText("XY position: " + str(mmc.getXYPosition()))

#     vbox = QtWidgets.QVBoxLayout()
#     vbox.addWidget(label)
#     window.setLayout(vbox)
#     window.show()
#     timer = QTimer()
#     timer.timeout.connect(update_label)
#     timer.start(10)
#     app.exec_()


def cv_liveview():
    mmc.enableStderrLog(False)
    mmc.enableDebugLog(False)
    mmc.setCircularBufferMemoryFootprint(500)
    mmc.setCameraDevice("Iris15")

    cv2.namedWindow("Live View")
    mmc.startContinuousSequenceAcquisition(1)
    mmc.setProperty("Shutter", "Volts", 5.0)

    while True:
        if mmc.getRemainingImageCount() > 0:
            img = mmc.getLastImage()
            img = scale_img(img, 0.3)
            disp_info(img)

            cv2.imshow("Live View", img)

        if cv2.waitKey(20) >= 0:
            break

    cv2.destroyAllWindows()
    mmc.stopSequenceAcquisition()
    mmc.setProperty("Shutter", "Volts", 0.0)

In [ ]:
cv_liveview()

In [ ]:
from PyQt5.QtWidgets import QApplication, QLabel, QMainWindow, QPushButton

%gui qt
app = QApplication([])
# win = QMainWindow()
label = QLabel("Hello World!")
# label1.setText("Hello")
# button = QPushButton('Test')
# win.setCentralWidget(button)
label.show()
app.exit(app.exec_())

In [ ]:
if not QtWidgets.QApplication.instance():
        app = QtWidgets.QApplication(sys.argv)
    else:
        app = QtWidgets.QApplication.instance()
main = MainWindow()
main.show()

In [ ]:
mmc.setShutterOpen("LightEngine", False)

In [ ]:
mmc.setShutterOpen("Turret1Shutter", True)

In [ ]:
liveview("Iris15", low=None, high=None)

In [ ]:
import cv2
import numpy as np

# import MMCorePy

# DEVICE = ['Camera', 'DemoCamera', 'DCam']
DEVICE = ["Iris15", "OpenCVgrabber", "OpenCVgrabber"]
# DEVICE = ['Camera', "BaumerOptronic", "BaumerOptronic"]

# mmc = MMCorePy.CMMCore()
mmc.enableStderrLog(False)
mmc.enableDebugLog(False)
mmc.setCircularBufferMemoryFootprint(100)
# mmc.loadDevice(*DEVICE)
# mmc.initializeDevice(DEVICE[0])
mmc.setCameraDevice(DEVICE[0])

cv2.namedWindow("Video")
mmc.startContinuousSequenceAcquisition(1)
mmc.setProperty("Shutter", "Volts", 5.0)
while True:
    #     rgb32 = mmc.getLastImage()
    if mmc.getRemainingImageCount() > 0:
        # rgb32 = mmc.popNextImage()
        img = mmc.getLastImage()
        # Efficient conversion without data copying.
        #         bgr = rgb32.view(dtype=np.uint8).reshape(
        #             rgb32.shape[0], rgb32.shape[1], 4)[..., :3]
        print(np.mean(img))
        cv2.imshow("Video", img)
    else:
        print("No frame")
    if cv2.waitKey(20) >= 0:
        break
cv2.destroyAllWindows()
mmc.stopSequenceAcquisition()
mmc.setProperty("Shutter", "Volts", 0.0)

In [ ]:
app = QtWidgets.QApplication.instance()
if app is None:
    app = QtWidgets.QApplication(sys.argv)
else:
    print("QApplication instance already exists: %s" % str(app))

window = QtWidgets.QWidget()
window.setGeometry(100, 100, 500, 300)
window.setWindowTitle("Image Viewer")

label = QtWidgets.QLabel()
label.setAlignment(Qt.AlignCenter)


def update_label():
    label.setText("XY position: " + str(mmc.getXYPosition()))


vbox = QtWidgets.QVBoxLayout()
vbox.addWidget(label)
window.setLayout(vbox)
window.show()
timer = QTimer()
timer.timeout.connect(update_label)
timer.start(10)
app.exec_()

In [ ]:
DEVICE = ["Camera", "DemoCamera", "DCam"]
# DEVICE = ['Camera', 'OpenCVgrabber', 'OpenCVgrabber']
# DEVICE = ['Camera', 'BaumerOptronic', 'BaumerOptronic']
OUT_VIDEO = "output.avi"


def set_mmc_resolution(mmc, width, height):
    """Select rectangular ROI in center of the frame."""
    x = (mmc.getImageWidth() - width) / 2
    y = (mmc.getImageHeight() - height) / 2
    mmc.setROI(x, y, width, height)


def CV_FOURCC(c1, c2, c3, c4):
    """Missed in cv2."""
    return (c1 & 255) + ((c2 & 255) << 8) + ((c3 & 255) << 16) + ((c4 & 255) << 24)


def main():
    """Looping in function should be faster then in global scope."""
    mmc = MMCorePy.CMMCore()
    mmc.enableStderrLog(False)
    mmc.enableDebugLog(False)
    mmc.setCircularBufferMemoryFootprint(1500)
    mmc.loadDevice(*DEVICE)
    mmc.initializeDevice(DEVICE[0])
    mmc.setCameraDevice(DEVICE[0])
    mmc.setProperty(DEVICE[0], "PixelType", "32bitRGB")

    cv2.namedWindow("MM controls")
    cv2.namedWindow("Video")
    if mmc.hasProperty(DEVICE[0], "Gain"):
        cv2.createTrackbar(
            "Gain",
            "MM controls",
            int(float(mmc.getProperty(DEVICE[0], "Gain"))),
            int(mmc.getPropertyUpperLimit(DEVICE[0], "Gain")),
            lambda value: mmc.setProperty(DEVICE[0], "Gain", value),
        )
    if mmc.hasProperty(DEVICE[0], "Exposure"):
        cv2.createTrackbar(
            "Exposure",
            "MM controls",
            int(float(mmc.getProperty(DEVICE[0], "Exposure"))),
            100,  # int(mmc.getPropertyUpperLimit(DEVICE[0], 'Exposure')),
            lambda value: mmc.setProperty(DEVICE[0], "Exposure", int(value)),
        )
    # set_mmc_resolution(mmc, 1280, 1024)
    # set_mmc_resolution(mmc, 640, 480)
    # set_mmc_resolution(mmc, 320, 240)
    # codecArr = 'X264'
    codecArr = "LAGS"  # Lagarith Lossless Codec
    fourcc = CV_FOURCC(
        ord(codecArr[0]), ord(codecArr[1]), ord(codecArr[2]), ord(codecArr[3])
    )
    out = cv2.VideoWriter(
        filename=OUT_VIDEO,
        fourcc=fourcc,  # '-1' Ask for an codec; '0' disables compressing.
        fps=20.0,
        frameSize=(mmc.getImageWidth(), mmc.getImageHeight()),
        isColor=True,
    )
    assert out.isOpened()
    mmc.startContinuousSequenceAcquisition(1)
    while True:
        if mmc.isBufferOverflowed():
            print("Buffer overflowed\a")  # Alert sound
            break
        start_time = time.time()  # FPS counter
        remcount = mmc.getRemainingImageCount()
        freecap = mmc.getBufferFreeCapacity()
        totalcap = mmc.getBufferTotalCapacity()
        print("%d frames, %d / %d = %.2f %%") % (
            remcount,
            freecap,
            totalcap,
            100.0 * freecap / float(totalcap),
        )
        print(mmc.getImageBufferSize())

        if remcount > 0:
            rgb32 = mmc.popNextImage()
            # rgb32 = mmc.getLastImage()
            bgr = rgb32.view(dtype=np.uint8).reshape(rgb32.shape[0], rgb32.shape[1], 4)[
                ..., :3
            ]
            cv2.imshow("Video", bgr)
            out.write(bgr)
        else:
            print("No frame")
        if cv2.waitKey(3) >= 0:
            break
        end_time = time.time()
        if end_time != start_time:
            # Prevent division by zero
            print("Cycle per second: %f") % (1.0 / (time.time() - start_time))

    # Take rest images from circular buffer.
    mmc.stopSequenceAcquisition()
    remcount = mmc.getRemainingImageCount()
    print("Remaining images in buffer: %d") % remcount
    for fn in xrange(remcount):
        rgb32 = mmc.popNextImage()
        bgr = rgb32.view(dtype=np.uint8).reshape(rgb32.shape[0], rgb32.shape[1], 4)[
            ..., :3
        ]
        print("Dump from buffer: %d") % (remcount - fn)
        out.write(bgr)

    out.release()
    mmc.reset()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()